<a href="https://colab.research.google.com/github/JuanManuelLopezP/Anal-tica/blob/main/Modelo_Arbol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Arboles de decisión**
0. Cargar librerias de referencia para el modelo


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #Graficador de cosas
from sklearn.tree import DecisionTreeClassifier  #Modelo

#Para no tener que cargar el archivo, se usarán 2 librerias
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


1. Se procede con la carga de los datos "Solicitantes crédito


In [ ]:
from sre_constants import ANY_ALL
nxl='/content/drive/MyDrive/Analítica/0. SolicitantesCrédito(USD)_Arboles.xlsx'
XDB=pd.read_excel(nxl, sheet_name=1)
XDB=XDB.dropna() #Elimina celdas vacias
XDB.head(100)

#Se procede con la selección de las variables de trabajo
XD=np.array(XDB[['Edad','Ingresos','Egresos','Monto (EAD)']]) #Datos entrada
print(XD)
yd=np.array(XDB['PreApr']) #Datos de salida
print(yd)

[[  38.         1356.144      1685.62266667  625.56223022]
 [  51.          286.016       364.462       140.03198442]
 [  37.          517.46325     629.20888889  284.56449156]
 ...
 [  38.          773.01975     672.91066667  594.94789369]
 [  43.          635.50175     780.69155556  305.58053861]
 [  37.         1419.26675    1169.83844444 1005.63687051]]
[1 1 0 ... 1 1 1]


2. Creación del modelo

In [ ]:
mar=DecisionTreeClassifier(criterion='gini',max_depth=4) #4 niveles o variables de entrada
mar=mar.fit(XD,yd)
ydp=mar.predict(XD)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(yd,ydp)
print(cm)

#Evaluamos modelo
VN=cm[0,0]
FP=cm[0,1]
FN=cm[1,0]
VP=cm[1,1]

Ex=(VN+VP)/(VN+VP+FN+FP)   #Que porcentage de los datos pronosticó correctamente
print("La exactitud alcanzada por el modelo es:",Ex)
se=VP/(FN+VP) #Como le fue con los positivos
print("La sensibildiad alcanzada por el modelo fue:",se)
esp=VN/(VN+FP) #Como le fue con los negativos
print("La especificidad alcanzada por el modelo fue:",esp)

[[2301  658]
 [ 644 2239]]
La exactitud alcanzada por el modelo es: 0.7771311194796303
La sensibildiad alcanzada por el modelo fue: 0.7766215747485259
La especificidad alcanzada por el modelo fue: 0.7776275768840825


3.¿Donde está el arbol?

In [ ]:
#Librerias para graficar el arbol
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data

vs=['Edad','Ingresos','Egresos','Monto (EAD)'] #Son titulos del arbol
dot_graph=export_graphviz(mar,feature_names=vs)
graph=graph_from_dot_data(dot_graph)
graph.write_png('arbol.png')

True

4. Pronóstico del solicitante

In [ ]:
XDB2=pd.read_excel(nxl, sheet_name=2)
XDB2=XDB2.dropna() #Elimina celdas vacias
XDB2.head(100)

XDB2=np.array(XDB2[['Edad','Ingresos','Egresos','Monto (EAD)']]) #Datos entrada
print(XDB2)

#Pronosticamos la Preaprobación
yd2=mar.predict(XDB2)
print(ydp2)
print("El número de preaporbados es:",np.sum(ydp2))
print("El número total de datos es:",len(ydp2))
print("El número de datos PreNegados es:",len(ydp2)-np.sum(ydp2))


[[  38.          383.67775     368.59866667  175.84723507]
 [  47.           34.1875       38.03444444   26.95587069]
 [  46.         1774.305      2116.72377778  705.43470796]
 ...
 [  32.          283.26575     170.26644444  225.30034584]
 [  35.          768.288       847.26444444  479.16423193]
 [  24.          487.598       376.18355556  424.283721  ]]
[0 0 1 ... 0 1 0]
El número de preaporbados es: 1260
El número total de datos es: 2536
El número de datos PreNegados es: 1276


**ANALISIS DE RESULTADOS**
De acuerdo con la base de datos suministrada y el nodo base del arbol de decisión, se puede observar un total de 5842 datos divididos en 2959 preapropabos y 2883 prenegados, esto representa el 51% y el 49% respectivamente. De acuerdo con el pronóstico realizado por el modelo, este alcanzó una exactitud cercana al 77%, valores que fueron similares a la sensibilidad que mide el comportamiento del modelo frente a la clasificación de los preaprovados, así como frente a la especificidad, indicador que mide el comportamiento del modelo frente a la clasificación de los prenegados.
Se destaca el único nodo puro "[PreNeg:10,PreApr:0, en donde los solicitantes de crédito deben de cumplir con la siguiente regla: SI ing>644 AND Egr<=685 AND Ing>376
AND Monto>=322.
Con respecto al pronóstico, se puede observar un total de 2536 individuos sin PreAprovación, de los cuales un total de 1260 pueden ser PreAprobados y un total de 1276 que pueden ser PreNegados

4 niveles